In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/"

'Colab Notebooks'     'Untitled Diagram (1).drawio'
 sample_solution.csv  'Untitled Diagram (2).drawio'
 test.csv	      'Untitled Diagram.drawio'
 train.csv	      'Untitled Diagram.drawio.png'


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/train.csv")

In [ ]:
data

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,1,3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
1,1,16,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
2,1,25,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
3,1,35,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
4,1,45,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13765196,1180945,38,9.0,33.0,19.5,25.5,36.5,33.0,20.5,28.0,36.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765197,1180945,42,9.0,33.0,21.0,30.5,37.0,36.5,22.0,33.5,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.636004
13765198,1180945,47,9.0,29.5,10.0,26.0,30.5,31.0,16.5,26.0,33.0,1.041667,NaN,1.025000,1.051667,1.75,NaN,0.750,3.0000,13.379990,NaN,NaN,13.379990,8.636004
13765199,1180945,52,9.0,19.0,NaN,15.5,26.5,19.0,NaN,16.5,26.5,NaN,NaN,NaN,1.051667,NaN,NaN,NaN,2.8125,NaN,NaN,NaN,NaN,8.636004


In [ ]:
train=data[data[:].notnull().all(1)]
train = train.drop(columns=['Expected'])

In [ ]:
# Среднее значение
mymean = train.mean(axis=0)
# Стандартное отклонение
mystd = train.std(axis=0)
train -= mymean
train /= mystd

In [ ]:
train

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th
6,-1.730214,-1.653082,-1.764987,-2.027488,-2.137413,-2.345488,-2.451912,-1.624559,-1.820328,-1.513511,-1.131028,0.394948,0.872967,0.552529,-0.424410,-0.033599,0.581523,-0.171791,-0.822728,0.333984,0.733073,-0.338742,-1.061764
9,-1.730214,-0.779645,-1.764987,-0.990108,-1.028839,-1.114821,-1.212121,-0.994261,-0.915443,-1.032068,-1.188719,0.320814,0.816740,0.552529,-0.251617,-0.159656,0.922629,-0.049742,-1.034499,0.102528,0.864761,-0.016213,-1.061764
10,-1.730214,-0.488500,-1.764987,-0.240889,-0.720901,-0.684088,-0.798857,-0.535863,-0.553489,-0.610805,-0.611807,0.394948,0.816740,0.552529,-0.424410,-0.159656,0.837352,-0.415888,-1.175680,-0.125666,0.864761,-0.338742,-0.810516
11,-1.730214,-0.197354,-1.764987,-1.681695,-1.275188,-1.299421,-1.330196,-1.452660,-1.036094,-1.212609,-1.419484,0.394948,0.816740,0.552529,-0.424410,0.155488,1.093181,0.072306,-1.034499,-0.585316,0.601386,-0.338742,-1.187390
12,-1.730214,0.093791,-1.764987,-0.471418,-0.412964,-0.561021,-0.739819,-0.363963,-0.251860,-0.430264,-0.669498,0.394948,0.816740,0.552529,-0.251617,-0.411772,0.496247,-0.293840,-1.105089,-0.357118,0.465937,-0.670482,-1.316604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13765002,1.755423,0.210250,-0.945791,-1.047741,-1.829476,-1.237888,-0.680782,-1.338060,-1.277397,-1.393150,-0.900263,0.394948,0.760514,0.552529,-0.424410,0.218516,1.093181,0.316404,-0.822728,-0.243023,0.733073,-0.016213,-1.061764
13765003,1.755423,0.559624,-0.945791,-1.624063,-1.891063,-1.791688,-1.212121,-0.994261,-0.855117,-0.911707,-1.131028,0.394948,0.760514,0.377387,-0.424410,0.596689,1.690116,1.170745,-0.328595,-0.011567,0.864761,0.960572,-0.810516
13765004,1.755423,0.908999,-0.945791,-1.566431,-1.459951,-1.607088,-1.507309,-0.822362,-0.915443,-0.971887,-1.131028,0.320814,0.816740,0.552529,-0.424410,0.344574,0.922629,0.072306,-0.963909,-0.125666,1.131892,0.306303,-1.065351
13765005,1.755423,1.258373,-0.945791,-0.932476,-1.336776,-1.422488,-1.507309,-0.994261,-1.036094,-1.032068,-1.015645,0.172546,0.704287,0.552529,0.093963,0.533660,0.837352,-0.049742,-0.752137,0.219885,0.733073,0.306303,-1.061764


In [ ]:
train.describe()

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th
count,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06,2.769088e+06
mean,-1.005507e-13,1.241574e-14,1.952397e-14,-1.077383e-13,-1.237997e-14,2.319070e-14,2.260570e-15,-7.575638e-15,2.200161e-14,1.184411e-14,-1.299678e-14,-3.422791e-11,-5.474002e-11,-1.372426e-10,3.562059e-10,-1.699251e-14,4.074514e-14,-4.457130e-14,-9.642849e-16,3.732864e-14,2.855122e-12,2.705135e-13,1.680694e-11
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.730214e+00,-1.711311e+00,-2.311118e+00,-4.909099e+00,-5.524723e+00,-4.991423e+00,-4.636305e+00,-4.718749e+00,-4.534985e+00,-4.341991e+00,-4.188663e+00,-1.717485e+01,-1.245271e+01,-2.992224e+01,-1.355658e+01,-8.353400e+00,-9.992751e+00,-9.935685e+00,-3.505165e+00,-1.861594e+01,-2.381311e+01,-1.565402e+01,-3.071756e+00
25%,-8.732342e-01,-8.378744e-01,-6.727255e-01,-7.019472e-01,-6.593139e-01,-6.840879e-01,-6.807818e-01,-7.077622e-01,-6.741402e-01,-7.311656e-01,-7.271891e-01,-1.239910e-01,-3.077897e-01,-4.983244e-01,-5.972019e-01,-5.378291e-01,-5.270703e-01,-6.599856e-01,-7.521372e-01,-4.712168e-01,-3.918994e-01,-3.387422e-01,-6.848894e-01
50%,2.585581e-02,-2.266689e-02,1.464705e-01,-6.799277e-02,-4.343924e-02,-6.875437e-02,-3.136759e-02,-7.746425e-02,-1.055762e-02,-6.918099e-02,-3.489425e-02,2.466805e-01,3.107022e-01,2.022459e-01,-4.244104e-01,-3.359872e-02,6.986453e-02,-1.717909e-01,-1.874136e-01,-1.156656e-02,2.063335e-01,-1.621252e-02,-2.146960e-01
75%,8.643610e-01,8.507697e-01,6.926012e-01,6.235939e-01,6.956103e-01,6.696459e-01,6.770843e-01,6.674333e-01,6.530250e-01,6.529840e-01,6.574006e-01,3.949480e-01,6.480606e-01,5.525288e-01,2.667581e-01,5.336605e-01,5.815229e-01,5.605012e-01,5.184908e-01,4.480787e-01,6.201990e-01,6.288328e-01,4.529083e-01
max,1.755423e+00,1.724206e+00,1.784863e+00,5.003643e+00,4.945145e+00,4.792381e+00,4.632607e+00,6.798513e+00,4.875823e+00,4.865613e+00,6.599598e+00,1.581095e+00,1.772591e+00,3.354809e+00,2.340258e+00,7.592886e+00,1.158218e+01,1.471815e+01,7.153993e+00,2.522023e+01,2.456274e+00,9.843805e+00,4.111574e+01


In [ ]:
y = train.iloc[:, 22]

In [ ]:
y

6          -1.061764
9          -1.061764
10         -0.810516
11         -1.187390
12         -1.316604
              ...   
13765002   -1.061764
13765003   -0.810516
13765004   -1.065351
13765005   -1.061764
13765006   -0.681302
Name: Kdp_5x5_90th, Length: 2769088, dtype: float64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=42)

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

In [ ]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(units=hp.Int('units_input', # Полносвязный слой с разным количеством нейронов
    min_value=128, # минимальное количество нейронов - 128
    max_value=1024, # максимальное количество - 1024
    step=32), input_shape=(X_train.shape[1],), activation='relu'))
  
  model.add(Dense(1))

  model.compile(optimizer="adam", loss='mse', metrics=['mae'])
  
  return model

In [ ]:
rm -Rf /content/test_directory

In [ ]:
tuner = RandomSearch(
  build_model, # функция создания модели
  objective='val_mae', # метрика, которую нужно оптимизировать   доля правильных ответов на проверочном наборе данных
  max_trials=5, # максимальное количество запусков обучения
  directory='test_directory' # каталог, куда сохраняются обученные сети
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': None}


In [ ]:
tuner.search(X_train, # Данные для обучения
             y_train, # Данные для обучения
             batch_size=250, # Размер мини-выборки
             epochs=10, # Количество эпох обучения
             validation_split=0.2, # Часть данных, которая будет использоваться для проверки
            verbose=1,)

Trial 5 Complete [00h 03m 31s]
val_mae: 0.0006803500582464039

Best val_mae So Far: 0.0003890393709298223
Total elapsed time: 00h 19m 01s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Objective(name='val_mae', direction='min')
Trial summary
Hyperparameters:
units_input: 224
Score: 0.0003890393709298223
Trial summary
Hyperparameters:
units_input: 928
Score: 0.0005151439690962434
Trial summary
Hyperparameters:
units_input: 576
Score: 0.0005986620089970529
Trial summary
Hyperparameters:
units_input: 864
Score: 0.0006803500582464039
Trial summary
Hyperparameters:
units_input: 960
Score: 0.0008732301648706198


In [ ]:
models = tuner.get_best_models(num_models=3)

In [ ]:
for model in models:
  model.summary()
  model.evaluate(X_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 224)               5376      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 225       
Total params: 5,601
Trainable params: 5,601
Non-trainable params: 0
_________________________________________________________________
28557/28557 [==============================] - 31s 1ms/step - loss: 2.6822e-07 - mae: 3.8870e-04

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 928)               22272     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 929       
Total params: 23,201
Trainable params: 23,201
Non-trainable params: 

In [ ]:
def build_model2(hp):
  model = Sequential()

  model.add(Dense(units=hp.Int('units_input', # Полносвязный слой с разным количеством нейронов
    min_value=512, # минимальное кол ичество нейронов - 128
    max_value=1024, # максимальное количество - 1024
    step=32), input_shape=(X_train.shape[1],), activation='relu'))
  
  model.add(Dense(units=hp.Int('units_hidden', min_value=128, max_value=600, step=32), activation='relu'))
  
  model.add(Dense(1))

  model.compile(optimizer="adam", loss='mse', metrics=['mae'])
  return model

In [ ]:
rm -Rf /content/test_directory2

In [ ]:
tuner2 = BayesianOptimization(
  build_model2, # функция создания модели
  objective ='val_mae', # метрика, которую нужно оптимизировать
  max_trials = 5, # максимальное количество запусков обучения
  directory ='test_directory2' # каталог, куда сохраняются обученные сети
)

In [ ]:
tuner2.search_space_summary()

Search space summary
Default search space size: 2
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': None}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': None}


In [ ]:
tuner2.search(X_train, # Данные для обучения
             y_train, # Данные для обучения
             batch_size=1000, # Размер мини-выборки
             epochs=20, # Количество эпох обучения
             validation_split=0.2, # Часть данных, которая будет использоваться для проверки
            verbose=1,)

Trial 5 Complete [00h 19m 44s]
val_mae: 0.0004893325385637581

Best val_mae So Far: 0.00045187119394540787
Total elapsed time: 01h 36m 16s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner2.results_summary()

In [ ]:
models2 = tuner2.get_best_models(num_models=3)

In [ ]:
for model in models2:
  model.summary()
  model.evaluate(X_test, y_test)
  print() 

In [ ]:
def build_model3(hp):
  model = Sequential()

  activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])

  model.add(Dense(units=hp.Int('units_input', # Полносвязный слой с разным количеством нейронов
                               min_value=512, # минимальное количество нейронов - 128
                               max_value=1024, # максимальное количество - 1024
                               step=32), input_shape=(X_train.shape[1],), activation=activation_choice))
  
  model.add(Dense(units=hp.Int('units_hidden', min_value=128, max_value=600, step=32), activation=activation_choice))
  
  model.add(Dense(1))

  model.compile(optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']), loss='mse', metrics=['mae'])

  return model

In [ ]:
rm -Rf /content/test_directory3

In [ ]:
tuner3 = BayesianOptimization(
  build_model3, # функция создания модели
  objective ='val_mae', # метрика, которую нужно оптимизировать
  max_trials = 5, # максимальное количество запусков обучения
  directory ='test_directory3' # каталог, куда сохраняются обученные сети
)

In [ ]:
!free -h

In [ ]:
tuner3.search_space_summary()

In [ ]:
tuner3.search(X_train, # Данные для обучения
             y_train, # Данные для обучения
             batch_size=32, # Размер мини-выборки
             epochs=10, # Количество эпох обучения
             validation_split=0.2, # Часть данных, которая будет использоваться для проверки
            verbose=1,)

In [ ]:
tuner3.results_summary()

In [ ]:
models3 = tuner3.get_best_models(num_models=3)

In [ ]:
for model in models3:
  model.summary()
  model.evaluate(X_test, y_test)
  print() 

In [ ]:
def build_model4(hp):

  model = Sequential()

  activation_choice = hp.Choice("activation", values=['relu', 'selu', 'elu'])
  
  model.add(Dense(units=hp.Int("units_input", min_value=32,
                               max_value=1024,
                               step=32),
                  activation=activation_choice, 
                  input_shape=(X_train.shape[1], )))
  
  model.add(Dropout(0.3))

  for i in range(hp.Choice('hidden_layers', values=[1,2,3])):
    model.add(Dense(units=hp.Int(f'layer_{i}_units_',
                                 min_value=32//(i+1),
                                 max_value=512//(i+1), 
                                 step=64//(i+1)),
                     activation=activation_choice))
  
  model.add(Dense(1))

  model.compile(optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']), loss='mse', metrics=['mae'])

  model.summary()
  return model

In [ ]:
tuner4 = BayesianOptimization(
  build_model4, # функция создания модели
  objective ='val_mae', # метрика, которую нужно оптимизировать
  max_trials = 5, # максимальное количество запусков обучения
  directory ='test_directory5' # каталог, куда сохраняются обученные сети
)

In [ ]:
tuner4.search_space_summary()

In [ ]:
tuner4.search(X_train, # Данные для обучения
             y_train, # Данные для обучения
             batch_size=32, # Размер мини-выборки
             epochs=10, # Количество эпох обучения
             validation_split=0.2, # Часть данных, которая будет использоваться для проверки
            verbose=1,)

In [ ]:
tuner4.results_summary()

In [ ]:
models4 = tuner4.get_best_models(num_models=3)

In [ ]:
for model in models4:
  model.summary()
  model.evaluate(X_test, y_test)
  print() 

In [ ]:
best_params = tuner4.get_best_hyperparameters()[0]
best_model = tuner4.hypermodel.build(best_params)
best_model.summary()

In [ ]:
best_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
predicted_y_tuner = best_model.predict(X_test)
predicted_y_tuner = np.round(predicted_y_tuner, 0)

predicted_y_tuner = np.reshape(predicted_y_tuner, (predicted_y_tuner.shape[0]))

СС_tuner = np.corrcoef(predicted_y_tuner, y_test)
СС_tuner = СС_tuner[0][1]
print(f'Коэффициент корреляции с истинными данными: {СС_tuner}')

In [ ]:
model = Sequential()
model.add(Dense(448, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=120, validation_split=0.2, verbose=1)

In [ ]:
print(model.summary())

In [ ]:
pred = model.predict(X_test)

In [ ]:
y_test = y_test.reset_index(drop=True)

In [ ]:
print("Предсказанное наблюдение  в конце часа:", pred[23][0], ", правильное  наблюдение  в конце часа:", y_test[23])

In [ ]:
vec = np.array([])
for i in np.arange(0, len(pred)):
  vec = np.append(vec, pred[i][0])
vec 

In [ ]:
sorted = np.sort(pred, )
sorted

In [ ]:
for i in np.arange(0, len(y_test)):
    print("Предсказанное наблюдение  в конце часа:", pred[i][0], ", правильное наблюдение  в конце часа:", y_test[i], 'разница: ', np.abs(pred[i][0] - y_test[i]))

In [ ]:
СС_tuner = np.corrcoef(vec, y_test)
СС_tuner = СС_tuner[0][1]
print(f'Коэффициаент корреляция с истинными данными: {СС_tuner}')